# Evaluate AI agents (Azure AI Agent Service) in Azure AI Foundry

## Objective


This sample demonstrates how to evaluate an AI agent (Azure AI Agent Service) on these important aspects of your agentic workflow:

- Intent Resolution: Measures how well the agent identifies the user’s request, including how well it scopes the user’s intent, asks clarifying questions, and reminds end users of its scope of capabilities.
- Tool Call Accuracy: Evaluates the agent's ability to select the appropriate tools, and process correct parameters from previous steps.
- Task Adherence: Measures how well the agent’s response adheres to its assigned tasks, according to its system message and prior steps.

For AI agents outside of Azure AI Agent Service, you can still provide th agent data in the two formats (either simple data or agent messages) specified in the individual evaluator samples:
- [Intent resolution](https://aka.ms/intentresolution-sample)
- [Tool call accuracy](https://aka.ms/toolcallaccuracy-sample)
- [Task adherence](https://aka.ms/taskadherence-sample)
- [Response Completeness](https://aka.ms/rescompleteness-sample)



## Time 

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
Creating an agent using Azure AI agent service requires an Azure AI Foundry project and a deployed, supported model. See more details in [Create a new agent](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal).

For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities.    

Important: Make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Prerequisite

Before running the sample:
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for AI-assisted evaluators, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) **PROJECT_NAME** - Azure AI Project Name
8) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name
9) **AGENT_MODEL_DEPLOYMENT_NAME** - The deployment name of the model for your Azure AI agent, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.

### Initializing Project Client

In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import FunctionTool, ToolSet

# Import your custom functions to be used as Tools for the Agent
from user_functions import user_functions

project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

AGENT_NAME = "Seattle Tourist Assistant"

# Add Tools to be used by Agent
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(tools=toolset)

### Create an AI agent (Azure AI Agent Service)

In [2]:
agent = project_client.agents.create_agent(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    toolset=toolset,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_mc3diJKGt4pjMKZq8IgQnSxb


### Create Thread

In [3]:
thread = project_client.agents.create_thread_and_run(agent_id=agent.id)
print(f"Created thread, ID: {thread.thread_id}")
print(thread)

Created thread, ID: thread_1n3TpNNjoXYjvNEJO7i4HswG
{'id': 'run_yMCCUKyavbsLWcm96b7JcWNo', 'object': 'thread.run', 'created_at': 1756162418, 'assistant_id': 'asst_mc3diJKGt4pjMKZq8IgQnSxb', 'thread_id': 'thread_1n3TpNNjoXYjvNEJO7i4HswG', 'status': 'in_progress', 'started_at': 1756162418, 'expires_at': 1756164218, 'cancelled_at': None, 'failed_at': None, 'completed_at': None, 'required_action': None, 'last_error': None, 'model': 'gpt-4.1', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'function', 'function': {'name': 'opening_hours', 'description': 'Fetches the opening hours of a tourist destination in Seattle.', 'parameters': {'type': 'object', 'properties': {'tourist_destination': {'type': 'string', 'description': 'The tourist destination to fetch opening hours for.'}}, 'required': ['tourist_destination']}, 'strict': False}}, {'type': 'function', 'function': {'name': 'get_user_info', 'description': 'Retrieves user information based on user ID.', 'parameters': {'typ

## Conversation with Agent
Use below cells to have conversation with the agent
- `Create Message[1]`
- `Execute[2]`

### Create Message[1]

In [5]:
# Create message to thread

MESSAGE = "Can you email me weather info for Seattle ?"

message = project_client.agents.messages.create(
    thread_id=thread.thread_id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_ilKefi1WWqvPF81bLRy8afHG


### Execute[2]

In [6]:
run = project_client.agents.create_thread_and_process_run(agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

Run finished with status: RunStatus.COMPLETED
Run ID: run_9wClwjmDueaEJtyMdpbzQi6n


### List Messages

In [7]:
for message in project_client.agents.messages.list(thread_id=thread.thread_id, order="asc"):
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

Role: MessageRole.AGENT
Content: Hi! How can I assist you today?
----------------------------------------
Role: MessageRole.USER
Content: Can you email me weather info for Seattle ?
----------------------------------------


# Evaluate

### Get data from agent

In [8]:
from azure.ai.evaluation import AIAgentConverter

# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

# Use the thread id associated with the run to ensure the run and thread match.
# Fallback to the previously created thread if the run does not have an associated thread id.
thread_id = run.thread_id if getattr(run, "thread_id", None) else thread.thread_id
run_id = run.id
file_name = "evaluation_data.jsonl"

# Get a single agent run data
evaluation_data_single_run = converter.convert(thread_id=thread_id, run_id=run_id)

# Run this to save thread data to a JSONL file for evaluation
# Save the agent thread data to a JSONL file
import json
evaluation_data = converter.prepare_evaluation_data(thread_ids=thread_id, filename=file_name)
print(json.dumps(evaluation_data, indent=4))

Class AIAgentConverter: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FDPAgentDataRetriever: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AIAgentDataRetriever: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


[
    {
        "query": [
            {
                "role": "system",
                "content": "You are a helpful assistant"
            }
        ],
        "response": [
            {
                "createdAt": "2025-08-25T22:53:52Z",
                "run_id": "run_9wClwjmDueaEJtyMdpbzQi6n",
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        "text": "Hello! How can I assist you today?"
                    }
                ]
            }
        ],
        "tool_definitions": [
            {
                "name": "opening_hours",
                "type": "function",
                "description": "Fetches the opening hours of a tourist destination in Seattle.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "tourist_destination": {
                            "type": "string",
               

### Setting up evaluator

We will select the following evaluators to assess the different aspects relevant for agent quality: 

- [Intent resolution](https://aka.ms/intentresolution-sample): measures the extent of which an agent identifies the correct intent from a user query. Scale: integer 1-5. Higher is better.
- [Tool call accuracy](https://aka.ms/toolcallaccuracy-sample): evaluates the agent’s ability to select the appropriate tools, and process correct parameters from previous steps. Scale: float 0-1. Higher is better.
- [Task adherence](https://aka.ms/taskadherence-sample): measures the extent of which an agent’s final response adheres to the task based on its system message and a user query. Scale: integer 1-5. Higher is better.


In [ ]:
from azure.ai.evaluation import (
    ToolCallAccuracyEvaluator,
    AzureOpenAIModelConfiguration,
    IntentResolutionEvaluator,
    TaskAdherenceEvaluator,
)
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)
# Needed to use content safety evaluators
azure_ai_project = {
    "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
    "project_name": os.environ["AZURE_AI_FOUNDRY_PROJECT_NAME"],
    "resource_group_name": os.environ["AZURE_RESOURCE_GROUP_NAME"],
}

intent_resolution = IntentResolutionEvaluator(model_config=model_config)

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)

task_adherence = TaskAdherenceEvaluator(model_config=model_config)

Class IntentResolutionEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ToolCallAccuracyEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class TaskAdherenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Run Evaluator

In [ ]:
from azure.ai.evaluation import evaluate, AzureAIProject

proj = AzureAIProject(
    subscription_id=os.environ["AZURE_SUBSCRIPTION_ID"],
    resource_group_name=os.environ["AZURE_RESOURCE_GROUP_NAME"],
    project_name=os.environ["AZURE_AI_FOUNDRY_PROJECT_NAME"],
)

response = evaluate(
    data=file_name,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
    },
    azure_ai_project=proj,
    output_path="./evaluation_results.json"

)
pprint(f'AI Foundary URL: {response.get("studio_url")}')

EvaluationException: (InternalError) The get 'aifoundry825233136833' workspace request failed with HTTP 404 - (ResourceNotFound) The Resource 'Microsoft.MachineLearningServices/workspaces/aifoundry825233136833' under resource group 'AIFoundry' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix

## Inspect results on Azure AI Foundry

Go to AI Foundry URL for rich Azure AI Foundry data visualization to inspect the evaluation scores and reasoning to quickly identify bugs and issues of your agent to fix and improve.

In [9]:
# alternatively, you can use the following to get the evaluation results in memory

# average scores across all runs
pprint(response["metrics"])

NameError: name 'pprint' is not defined